O primeiro passo é criar os dados que serão explorados. No caso, criaremos três arrays: um com os códigos dos produtos, um com os pesos dos produtos, e outro com os preços dos produtos.

In [ ]:
import numpy as np
codProduto = np.arange(1,11)
print("Códigos dos produtos:", codProduto)
peso = np.array([5,8,2,10,7,1,6,20,1,9])
print("Pesos dos produtos:", peso)
preco = np.array([20,440,450,600,900,1000,30,100,500,800])
print("Preços dos produtos:", preco)

print("CodigoProduto    Peso    Preço")
for i in range(10):
  print(codProduto[i],"              ",peso[i],"     ", preco[i])

Códigos dos produtos: [ 1  2  3  4  5  6  7  8  9 10]
Pesos dos produtos: [ 5  8  2 10  7  1  6 20  1  9]
Preços dos produtos: [  20  440  450  600  900 1000   30  100  500  800]
CodigoProduto    Peso    Preço
1                5       20
2                8       440
3                2       450
4                10       600
5                7       900
6                1       1000
7                6       30
8                20       100
9                1       500
10                9       800


Uma vez que geramos os dados, vamos configurar os parâmetros do algoritmo genético.
* 1 - Definir a capacidade máxima da mochila (35kg)
* 2 - Definir a quantidade de genes do cromossomo (10)
* 3 - Definir o tamanho da população inicial 

In [ ]:
capacidadeMochila = 35
tamanhoCadeiaGenetica = 10
tamanhoPopulacao = 5
np.random.seed(2)
populacaoInicial = np.random.randint(0,2,size = (tamanhoPopulacao,tamanhoCadeiaGenetica))
print("A população inicial, gerada aleatoreamente foi: \n",populacaoInicial)

A população inicial, gerada aleatoreamente foi: 
 [[0 1 1 0 0 1 0 1 0 1]
 [0 1 1 1 1 1 1 1 0 0]
 [0 0 1 1 1 0 0 0 1 1]
 [1 0 0 1 0 0 1 1 1 0]
 [0 0 0 1 1 1 1 0 0 0]]


Uma vez que a população inicial foi gerada, temos que criar a função que calculará a aptidão de cada cromossomo.


*   Se a combinação de produtos do cromossomo não couber na mochila, a aptidão será ZERO
*   Se a combinação couber na mochila, a aptidão será a soma dos preços dos produtos contemplados no cromossomo



In [ ]:
def calculaAptidao(populacao):
  aptidao = np.zeros(tamanhoPopulacao)
  #print(aptidao)
  for i in range(tamanhoPopulacao):
    somaPeso = np.sum(populacao[i]*peso)
    if somaPeso > capacidadeMochila:
      aptidao[i] = 0
    else:
      aptidao[i] = np.sum(populacao[i]*preco)
  return aptidao 

In [ ]:
aptidaoCalculada = calculaAptidao(populacaoInicial)
aptidaoCalculada

array([   0.,    0., 3250.,    0., 2530.])

Uma vez que sabemos a aptidão de cada cromossomo da população, vamos desenvolver uma função para selecionar os cromossomos para reprodução. No nosso caso, utilizaremos a estratégia elitista, onde os dois cromosssomos mais aptos da população serão selecionados para reprodução.

In [ ]:
def selecionaPais(populacao, aptidao):
  numPais = 2
  pais = np.zeros((numPais, tamanhoCadeiaGenetica))
  aptidao_copy = np.copy((aptidao))
  for i in range(numPais):
    posicaoCromossomoMaiorAptidao = np.argmax(aptidao_copy)
    #print(posicaoCromossomoMaiorAptidao)
    pais[i,:] = populacao[posicaoCromossomoMaiorAptidao,:]
    #print(pais[i])
    aptidao_copy[posicaoCromossomoMaiorAptidao] = -1
    #print(aptidao_copy)
  return pais


In [ ]:
pais = selecionaPais(populacaoInicial,aptidaoCalculada)
pais

array([[0., 0., 1., 1., 1., 0., 0., 0., 1., 1.],
       [0., 0., 0., 1., 1., 1., 1., 0., 0., 0.]])

Uma vez que os pais foram selecionados para reprodução, é hora de desenvolvermos uma função que fará o crossover (cruzamento). No caso, utilizaremos a técnica de UM PONTO.

In [ ]:
def crossover(pais):
  numFilhos = 2
  filhos = np.zeros((numFilhos,tamanhoCadeiaGenetica))

  posicaoCrossover = 3

  filhos[0,posicaoCrossover:] = pais[0,posicaoCrossover:]
  filhos[0,0:posicaoCrossover] = pais[1,0:posicaoCrossover]

  filhos[1,posicaoCrossover:] = pais[1,posicaoCrossover:]
  filhos[1,0:posicaoCrossover] = pais[0,0:posicaoCrossover]

  return filhos

In [ ]:
filhos = crossover(pais)
filhos

array([[0., 0., 0., 1., 1., 0., 0., 0., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 0., 0., 0.]])

Chegou a hora de implementarmos a função de mutação.

In [ ]:
import random as rd
def mutacao(filhos):
  txMutacao = 0.2
  posicaoCromossomoMutacao = int(rd.random() * 10)
  mutantesCandidatos = np.copy(filhos)
  for i in range(2):
    valRandomico = rd.random()
    if valRandomico >= txMutacao:
      print("Filho ", i, "não sorteado para mutação")
      continue
    else:
      print("Filho ", i, "sorteado para mutação")
      if mutantesCandidatos[i,posicaoCromossomoMutacao] == 0:
        mutantesCandidatos[i,posicaoCromossomoMutacao] = 1
      else:
        mutantesCandidatos[i,posicaoCromossomoMutacao] = 0
  return mutantesCandidatos

In [ ]:
mutantes = mutacao(filhos)
mutantes

Filho  0 não sorteado para mutação
Filho  1 não sorteado para mutação


array([[0., 0., 0., 1., 1., 0., 0., 0., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 0., 0., 0.]])

Ao executar o algoritmo genético completo, será necessário ter uma função que calcula a aptidão de UM cromossomo específico.

In [ ]:
def calculaAptidaoIndividual(cromossomo):
  somaPeso = np.sum(cromossomo * peso)
  if somaPeso > capacidadeMochila:
    aptidao = 0
  else:
    somaCusto = np.sum(cromossomo * preco)
    aptidao = somaCusto
  return aptidao

In [ ]:
calculaAptidaoIndividual(populacaoInicial[0])

0

Para termos uma população sem cromossomos iguais, vamos criar uma função que avaliará se o cromossomo filho já está na população.

In [ ]:
def inPopulacao(cromossomo, populacao):
  for element in populacao:
    if np.array_equal(cromossomo, element):
      return False
  return True

Agora que todas as funções necessárias para o algoritmo genético foram criadas, vamos partir para a aplicação da teoria por completo. No nosso caso, utilizaremos como critério de parada o número de gerações.


In [ ]:
numGeracoes = 500
populacaoAtual = np.copy(populacaoInicial)

for i in range(numGeracoes):
  #1. Verificar a aptidão de cada cromossomo da população
  aptidaoPopulacaoAtual = calculaAptidao(populacaoAtual)
  #2. Selecionar os pais que farão o crossover
  pais = selecionaPais(populacaoAtual,aptidaoPopulacaoAtual)
  #3. Realizar o crossover
  filhos = crossover(pais)
  #4. Verificar se algum filho será sorteado para mutação
  filhos = mutacao(filhos)

  #5.Agora que os filhos forma gerados, vamos armaznear em uma variável
  # a posição do cromossomo da população atual com menor aptidão
  posicaoCromossomoMenorAptidao = np.argmin(aptidaoPopulacaoAtual)

  #6. Uma vez que armazenamos a posição do cromossomo com menor aptidão,
  # vamos avaliar se algum filho gerado é mais apto do que o cromossomo com
  # menor aptidão
  if calculaAptidaoIndividual(filhos[0,:])>calculaAptidaoIndividual(populacaoAtual[posicaoCromossomoMenorAptidao,:]) and (inPopulacao(filhos[0],populacaoAtual)):
    populacaoAtual[posicaoCromossomoMenorAptidao,:] = filhos[0,:]

  #7. Uma vez que a população pode ter sido atualizada em decorrência do filho 0
  # ser mais apto do que o cromossomo de menor aptidão. É necessário verificar
  # novamente o cromossomo de menor aptidão.
  aptidaoPopulacaoAtual = calculaAptidao(populacaoAtual)
  posicaoCromossomoMenorAptidao = np.argmin(aptidaoPopulacaoAtual)

  if calculaAptidaoIndividual(filhos[1,:])>calculaAptidaoIndividual(populacaoAtual[posicaoCromossomoMenorAptidao,:]) and inPopulacao(filhos[1],populacaoAtual):
    populacaoAtual[posicaoCromossomoMenorAptidao,:] = filhos[1,:]

  aptidaoPopulacaoAtual = calculaAptidao(populacaoAtual)
  print("Aptidao dos cromossomos após o crossover:", aptidaoPopulacaoAtual)
  print("Cromossomos da população após o crossover:", populacaoAtual)
  print("############################################################")
  print("############################################################")

Filho  0 não sorteado para mutação
Filho  1 não sorteado para mutação
Aptidao dos cromossomos após o crossover: [2800. 2980. 3250.    0. 2530.]
Cromossomos da população após o crossover: [[0 0 0 1 1 0 0 0 1 1]
 [0 0 1 1 1 1 1 0 0 0]
 [0 0 1 1 1 0 0 0 1 1]
 [1 0 0 1 0 0 1 1 1 0]
 [0 0 0 1 1 1 1 0 0 0]]
############################################################
############################################################
Filho  0 não sorteado para mutação
Filho  1 sorteado para mutação
Aptidao dos cromossomos após o crossover: [2800. 2980. 3250. 2950. 2530.]
Cromossomos da população após o crossover: [[0 0 0 1 1 0 0 0 1 1]
 [0 0 1 1 1 1 1 0 0 0]
 [0 0 1 1 1 0 0 0 1 1]
 [0 0 1 1 1 1 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0]]
############################################################
############################################################
Filho  0 sorteado para mutação
Filho  1 não sorteado para mutação
Aptidao dos cromossomos após o crossover: [2800. 2980. 3250. 2950. 3270.]
Cromossomos da